**Run the codes in this sequence: A, B, C, D, E, F, G, H, I, J, K.**

# (A) Import libraries

In [1]:
# os.chdir(r'C:/Users/UWAdmin/Desktop/TEST2/line codes')

from pymeasure.instruments.lighthousephotonics import Sprout #import laser codes
laser_power = Sprout('COM4')
laser_power.adapter.connection.baud_rate = 19200
laser_power.adapter.connection.read_termination = '\r'
laser_power.adapter.connection.write_termination = '\r'
laser_power.power = 0.01


C:\Users\UWAdmin\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


In [2]:
import time 
import rpy2.robjects as robjects

from newportxps import NewportXPS # motion controller
from thread import MotionThread, MirrorThread

import os
import numpy as np
import pandas as pd
from write_utils import write_data_files, get_mean, get_move_y, duplicate_to_dataset

from flipper import mirror 
from spectra import capture_photo

# (B) Activate motion controller

In [3]:
XPSD_REMOTE_IP = '192.168.254.254'

xps = NewportXPS(XPSD_REMOTE_IP)
print(xps.status_report())

#pos_all() to get position of all axes
def pos_all():
    for sname, _ in xps.stages.items():
        print('{}: {}'.format(sname, xps.get_stage_position(sname)))
        

xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()

# XPS host:         192.168.254.254 (XPS-1B81)
# Firmware:         XPS-D-N13019
# Current Time:     Tue Apr 19 07:20:18 2022
# Last Reboot:      Thu Mar  3 03:40:02 2022
# Trajectory Group: XYZ
# Groups and Stages
XYZ (multipleaxesinuse), Status: Ready state from motion
   XYZ.X (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK
   XYZ.Y (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK
   XYZ.Z (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK


In [4]:
# xps.kill_group('XYZ')
# xps.initialize_allgroups()
# xps.home_allgroups()
xps.move_stage('XYZ.X', 2.5)
xps.move_stage('XYZ.Y', 6.5)
xps.move_stage('XYZ.Z', 1.75)
pos_all()

XYZ.X: 2.5000048512
XYZ.Y: 6.5000338524
XYZ.Z: 1.7498334816


In [5]:
mirror("off")

MIRROR DOWN


## (C) Switch on the spectrometer

In [6]:
#Before running this code check that AddInProcess.exe is running (closed if open)
#Close anything related to Light Field

capture_photo("start",2,1,0) # the values 2,1,0 don't matter, they are later used for indexing tasks

## (D) Set chamber pressure
#Used to input new Sample
#Move this before auto focusing

In [3]:
from pressure import close_valve,open_valve,close_all,current_pressure,gopr, to_ambient, to_vacuum, quick_fill
current_pressure()

C:\Users\UWAdmin\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


26

In [9]:
t = 2 # in minutes
to_ambient()
time.sleep(1)
to_ambient()
to_vacuum(t)
quick_fill()
time.sleep(1)
to_vacuum(t)


1

# (E) To make sure that the material gets laser beam of max. intensity , focusing of z axis needs to be properly adjusted. With LightField running, use the following code to find z, enter 0 and confirm with yes to end.

In [8]:
mirror('on')

#Iterate for input to find z value
print("Enter z value (enter 0 when done):")
z=''
while z != 0:
    if z == '':
        z = float(input(':'))
    print(z)
    xps.move_stage('XYZ.Z', z)
    xps.move_stage('XYZ.X', 5)
    xps.move_stage('XYZ.Y', 7)
    z = float(input(':'))
    
# mirror('on')
pos_all()
while input('Is LightField Scan Stopped?:(yes/no)')!='yes':
    continue

MIRROR UP
Enter z value (enter 0 when done):
1.45
XYZ.X: 4.999992003
XYZ.Y: 7.000024203
XYZ.Z: 1.4500056456


# (F) Prepare initial dataset

## MAKE SURE THE BOUNDS ARE COMMENSURATE WITH R CODE PARAM SET

In [4]:
from pathlib import Path
path = Path(r'c:\\Users\\UWAdmin\\Desktop\\_pyControl\\campaigns')
# write_data_files(path, nr_random_lines=1, series=1, batch=True)

# (G) Define coordinate details

**Takes input for**  
  
-number of lines

-starting position of x

-starting position of y

-vertical gap/step size

In [5]:
lines, start_y, step_y = 7, 6.5, 1
move_y = get_move_y(lines, start_y)
print("Lines will be made at following y axis:", move_y,"\n")

Lines will be made at following y axis: [6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5] 



# (K) Begin the experiment

In [6]:
robjects.r('''
  experiment<-function(s){

  presToR <- function (x) {(x - 50) / 10}
  presToExp <- function (x) {x * 10 + 50}

  if(s==1){ #BO called first time to initialize and propose first point
  require(XML)

    presToR <- function (x) {(x - 50) / 10}
    presToExp <- function (x) {x * 10 + 50}

    suppressWarnings({suppressMessages({library(mlrMBO)
                                        library(ggplot2)
                                        library(doMC)
                                      })})

    ps = makeParamSet(
      makeIntegerParam("power", lower = 5, upper = 1190),
      makeIntegerParam("time", lower = 1050, upper = 7500),
      makeIntegerParam("pressure", lower = 1, upper = 30, trafo = function(x) x * 10 + 50)
    )

    ctrl = makeMBOControl(y.name = "resistance", propose.points = 14L)
    ctrl = setMBOControlInfill(ctrl, opt = "focussearch", 
                              opt.focussearch.maxit = 10,
                              opt.focussearch.points = 10000,
                              crit = makeMBOInfillCritEI())

    ctrl = setMBOControlMultiPoint(ctrl, method = "cl", cl.lie = min)
    ctrl = setMBOControlTermination(ctrl, iters = 1)

    data = read.csv("data.csv", colClasses = c(NA, NA, NA, "NULL", NA))
    data$pressure <- presToR(data$pressure)

    suppressMessages({opt.state = initSMBO(par.set = ps, 
                                          design = data,
                                          control = ctrl, 
                                          minimize = TRUE, 
                                          noisy = TRUE)})
    print("Proposed parameters:")
    prop = suppressWarnings({proposePoints(opt.state)})
    print(prop$prop.points)
    print(prop$crit.components)
    saveRDS(prop,'./prop.rds')
    saveRDS(opt.state,'./opt.state.rds')
    dummy_ratio = 0
    dummy_resistance = 0
    cat("Expected value (upper bound):\n")
    cat(paste(prop$crit.components$mean, " (",
              prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))

    prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)

    x <- data.frame(prop$prop.points, dummy_ratio, dummy_resistance,
                    prop$crit.components$mean,
                    prop$crit.components$mean + prop$crit.components$se, 
                    2*prop$crit.components$mean + prop$crit.components$se)

    x2 <- c(prop$prop.points)

    write.table(x2, file = "data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

    write.table(x, file = "plot_data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

  }
  else{
    
    presToR <- function (x) {(x-50)/10}
    presToExp <- function (x) {x*10+50}

    prop=readRDS('./prop.rds')
    opt.state=readRDS('./opt.state.rds')
    library(mlrMBO)
    library(R.utils)
    print(prop$crit.components)
    print(prop$prop.points)

    print("Proposed parameters:")
    l2keep <- 14
    nL <- countLines("data.csv")
    s <- read.csv("data.csv", header=FALSE, skip=nL-l2keep)['V5']
    updateSMBO(opt.state, x = prop$prop.points, y = as.list(s$V5))
    prop <- suppressWarnings({proposePoints(opt.state)})
    print(prop$prop.points)
    print(prop$crit.components)
    saveRDS(prop,'./prop.rds')
    saveRDS(opt.state,'./opt.state.rds')

    dummy_ratio = 0
    dummy_resistance = 0
    cat("Expected value (upper bound):\n")
    cat(paste(prop$crit.components$mean, " (",
              prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))

    prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)

    x <- data.frame(prop$prop.points, dummy_ratio, dummy_resistance,
                    prop$crit.components$mean,
                    prop$crit.components$mean + prop$crit.components$se, 
                    prop$crit.components$mean + prop$crit.components$mean + prop$crit.components$se)
    x2<-c(prop$prop.points)

    write.table(x2, file = "data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

    write.table(x, file = "plot_data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

  }}

  ''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x0000019C65200C08> [RTYPES.CLOSXP]
R classes: ('function',)

In [7]:
campaign_path = path / "2022-04-18-BATCH-1"
os.chdir(campaign_path)

d = pd.read_csv("data.csv")
rsum = robjects.r['experiment']

_NR_LINES_IN_SAMPLE = 14 # don't change this, best either 7 or 14

if len(d['resistance']) == _NR_LINES_IN_SAMPLE:
    print(f"First batch proposal of {_NR_LINES_IN_SAMPLE} lines")
    rsum((1))
    duplicate_to_dataset(nr_proposed_lines=_NR_LINES_IN_SAMPLE, prepattern=False)

elif len(d['resistance']) < _NR_LINES_IN_SAMPLE:
    print ("Not enough data points in dataset. Exiting.")

else:
    print(f"Further batch proposals of {_NR_LINES_IN_SAMPLE} lines")
    proposed = d['resistance'].iloc[-_NR_LINES_IN_SAMPLE:]
    rsum((list(proposed)))
    duplicate_to_dataset(nr_proposed_lines=_NR_LINES_IN_SAMPLE)


Further batch proposals of 14 lines


R[write to console]: Loading required package: mlr

R[write to console]: Loading required package: ParamHelpers

R[write to console]: Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

R[write to console]: Loading required package: smoof

R[write to console]: Loading required package: checkmate

R[write to console]: Loading required package: R.oo

R[write to console]: Loading required package: R.methodsS3

R[write to console]: R.methodsS3 v1.8.1 (2020-08-26 16:20:06 UTC) successfully loaded. See ?R.methodsS3 for help.

R[write to console]: R.oo v1.24.0 (2020-08-26 16:11:58 UTC) successfully loaded. See ?R.oo for help.

R[write to console]: 
Attaching package: 'R.oo'


R[write to console]: The following object is masked from 'package:R.methodsS3':

    throw


R[write to console]: The follo

         se      mean
1  929.4292 1623.2710
2  912.6779 1515.4984
3  905.8824 1442.2436
4  864.0528 1305.3174
5  838.8018 1236.3487
6  833.4613 1214.2246
7  808.4642 1165.5809
8  776.8364 1120.4417
9  776.8190 1106.8540
10 733.9176 1061.4488
11 717.9538 1052.5725
12 705.7178 1043.3130
13 685.8191 1024.0287
14 866.2571  554.7942
     power time pressure
40     914 1600        8
23     907 1555        6
728    902 1369        3
39     896 1619        4
141    893 1637        3
13     889 1474        1
220    887 1464        1
401    886 1576        1
1476   882 1295        1
402    881 1637        1
226    883 1591        1
199    881 1554        1
61     877 1648        1
26     241 3988        8
[1] "Proposed parameters:"
     power time pressure
500    508 3988       19
1550   390 4603       30
7373   969 7328       14
7411  1079 6331        3
2552   452 3925       19
299    453 3934       19
95     457 3918       19
110    442 3960       18
495    449 3956       19
49     931 3957   

In [8]:
col = 4
TOTAL_LINES = len(move_y)
for move_line in range(TOTAL_LINES):
        # print("For this print this is line #: ",move_line)
        # print("Experiment Line number: " ,move_line)
        
        save_line = col*TOTAL_LINES + move_line
        steps = save_line * 9
        print (f"before: {move_line}, {save_line}, {steps}")
        
        # print (f"after : {move_line}, {save_line}, {steps}")

before: 0, 28, 252
before: 1, 29, 261
before: 2, 30, 270
before: 3, 31, 279
before: 4, 32, 288
before: 5, 33, 297
before: 6, 34, 306


In [39]:
SPOTS_MEASURED = 9
save_line, steps = 34, 306
get_mean(steps, save_line, target='ratio')

c:\Users\UWAdmin\Desktop\_pyControl\campaigns\2022-04-18-BATCH-1


4.1747836445

In [ ]:
def main_batch_experiment(startx: float, move_y: list, col: int) -> None:
    """Starts the experiment. Currently works ok with equal length of col1 and col2 lines.
       startx: Starting position on the x-axis
       starty: Starting position on the y-axis
       step_y: The distance between lines on the y-axis
       move_y: List of positions on the y-axis
       col_no: Defines the column number"""

    # line_name_start = col_no*len(move_y) #0,1,   # What the first line # will be named
    # line_name_end = line_name_start + len(move_y) #What the last line # will be named
    
    import pandas as pd
    print(f'LOADING DATA.CSV FOR COLUMN: {col}') 
    df2 = pd.read_csv('data.csv')   # to avoid pd referenced before assignment
    print (df2)
    
    print("BATCH EXPERIMENT IS STARTING\n")
    import time
    time.sleep(5)
    # cc = 0
    # steps = 0
    MIRROR_MOVE_TIME = .5 # in seconds
    LINE_LENGTH = 2 # in mm
    VACUUM_TIME = 1 # in minutes

    #set this to zero!!!!!!!
    #starting_line = 4
    STARTING_LINE = 0
    SPOTS_MEASURED = 9
    TOTAL_LINES = len(move_y) #+STARTING_LINE
    TARGET = 'GD'
    

    print(f"Here's the number of total lines:{TOTAL_LINES}")
    
    for move_line in range(STARTING_LINE,TOTAL_LINES):
        print(f"====== MOVE LINE: {move_line} ======")
        
        save_line = col * TOTAL_LINES + move_line
        steps = save_line * SPOTS_MEASURED
        print(f"====== SAVE LINE: {save_line} ======")

        # move the axes to their start position
        xps.move_stage('XYZ.Y',move_y[move_line])
        pos_all()
        print("\n\n")
        df2 = pd.read_csv('data.csv')
        plot_data_df = pd.read_csv('plot_data.csv')
        print(f"\n data.csv file before index {save_line}:\n")
        print(df2)
        print(f"\n plot_data.csv file before index {save_line}:\n")
        print(plot_data_df,'\n')
        # line = df2['ratio'].count() #find in data.csv what line number we're on
        
        # set time
        time_of_file = df2['time'][save_line]
        print(f"\nCurrent patterning time is: {time_of_file} ms")
        
        # vacuum-infill
        print("Vacuum-infill in progress...")
        t = 1 # in minutes
        to_ambient()
        time.sleep(1)
        to_ambient()
        to_vacuum(t)
        quick_fill()
        time.sleep(1)
        to_vacuum(t)

        # set pressure
        pressure_of_file=df2['pressure'][save_line]
        for j in range(3):
            gopr(pressure_of_file)
            close_all()
            time.sleep(5)
        print(f"\n Pressure is now :{pressure_of_file}\n")

        xps.define_line_trajectories(start=float(startx),
                                 stop=float(startx)+LINE_LENGTH,
                                 step=0.01,
                                 scantime=(time_of_file/1000),
                                 axis='X')
        # downloads trajectory defined above
        xps.download_trajectory('foreward.trj')
        import pandas as pd
        d = pd.read_csv('foreward.trj', header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        print(d)

        total_time = np.sum(d['ramptime'])
        mirror_sleep = d['ramptime'][0] - MIRROR_MOVE_TIME
        print(f'Total time: {total_time}, Mirror_sleep: {mirror_sleep} in MAIN')

        # power will be set to assigned value, make sure mirror is OFF (!) \
        # and power is given enough time to reach it's value

        power_of_file = df2["power"][save_line]


    ###########################

        # print("#############STARTING PRE-MEASUREMENT ###########\n")
        # check=[float(startx) + .2*LINE_LENGTH, \
        #         float(startx) + .4*LINE_LENGTH, \
        #         float(startx) + .6*LINE_LENGTH]
        # # mm=0
        # for spot in range(3):
        #     print(f"########## spot: {spot} ##############")
        #     xps.move_stage('XYZ.X',check[spot])
        #     print(pos_all())

        #     for iii in range(3):
        #         print(f"############## iteration: {iii} ################")
        #         mirror("on")
        #         #writing G/D of 3 spots in 3 lines inside dataset.csv
        # #         capture_photo("on",spot,line)
        #         GD=capture_photo("first",spot,line,iii)

        #         mirror("off")
        #         time.sleep(5)

    ############################

    # PATTERN EXPERIMENT


        print("Power is now: {} mW".format(power_of_file))
        xps.move_stage('XYZ.X',(float(startx) - d['rampdist'][0]))
        pos_all()
        time.sleep(15)
        print("\n\n")
        mirror("off")
        a=power_of_file
        laser_power.power = (a/0.0994)/1000
        time.sleep(15)
        
        
        print("###### Starting pattern in 5 seconds! #########")
        ##Draw Lines
        time.sleep(10)
        
        import sys
        motion_thread = MotionThread()
        mirror_thread = MirrorThread()

        motion_thread.start()
        mirror_thread.start()

        motion_thread.join()
        mirror_thread.join()

        time.sleep(10)
        print("\n Job done\n")

        ##raman spectra analysis
        laser_power.power=0.01
        time.sleep(15)

    ########################################

    # post patterning raman analysis

        check=[float(startx) + .2*LINE_LENGTH, \
                float(startx) + .4*LINE_LENGTH, \
                float(startx) + .6*LINE_LENGTH]
        # mm=0
        for spot in range(3):
            print(f"############### SPOT {spot} ##################")
            xps.move_stage('XYZ.X',check[spot])
            print(pos_all())

            for iii in range(3):
                print(f"############# iteration {iii} ###############")
                mirror("on")
                #writing G/D of 3 spots in 3 lines inside dataset.csv
        #         capture_photo("on",spot,line)
                print(f"Measuring patterned line for {TARGET}")
                GD = capture_photo("on", spot, save_line, iii, target=TARGET)

                mirror("off")
                time.sleep(5)      

        if save_line <= TOTAL_LINES:
            print(f"############# LINE {save_line} <= {TOTAL_LINES}, CURRENT_STEP={steps}, TAKING MEAN: #############")
            get_mean(steps, save_line)

In [75]:
import sys

# path=r'C:\Users\UWAdmin\Desktop\TEST2\Campaigns\2022-04-11-Sample1'
# os.chdir(path)

first_column = int(input("First column number:"))
first_column_verify = int(input("Verify first column number:"))
while first_column !=first_column_verify:
    first_column = int(input("Error, numbers don't match for first column. Re-enter first column number:"))
    first_column_verify = int(input("Verify first column number:"))

second_column = int(input("Second column number:"))
second_column_verify = int(input("Verify second column number:"))

while second_column !=second_column_verify:
    second_column = int(input("Error, numbers don't match for second column. Re-enter second column number:"))
    second_column_verify = int(input("Verify second column number:"))


intensity_check =capture_photo('adjust',0,0,0)
print("---Final Verification of Batch Experiment---")
print("Current Pressure: ", current_pressure())
pos_all()
print("Checked Intesity: ",intensity_check)
print(f"Print to columns: {first_column} and {second_column}")



verify_start = input("Do you approve pressure, intensity, and column numbers (yes/no)")
if verify_start == "yes":
    "Pressure, intensity, and column numbers have been approved. Starting experiment..."
    startx1 = 2.5
    main_batch_experiment(startx1, move_y, col=first_column)

    startx2 = 5.5
    main_batch_experiment(startx2, move_y, col=second_column)
else:
    sys.exit()





MIRROR UP
 
MIRROR DOWN
---Final Verification of Experiment---
Current Pressure:  1
XYZ.X: 4.999992003
XYZ.Y: 7.0000065036
XYZ.Z: 1.1800012986
Current Intesity:  669.6666666666666
Print to columns: 2 and 3
LOADING DATA.CSV
    power  time  pressure     ratio
0     159  1958       100  0.857771
1     238  3549       220  1.333680
2    1079  2802       230  1.242668
3     724  2631       210  1.190802
4     481  3995       260  0.808519
5     839  4650       220  0.754259
6     278  3269       220  0.805314
7     823  4957       120  3.001912
8     887  5000       120  1.276346
9     816  4954       120       NaN
10    823  4957       120       NaN
11    823  4957       120       NaN
12    823  4957       120       NaN
13    823  4957       120       NaN
14    823  4957       120       NaN
EXPERIMENT IS STARTING

Here's the number of total lines:7
====== MOVE LINE: 0 ======
====== SAVE LINE: 14 ======
XYZ.X: 4.999992003
XYZ.Y: 6.5001754476
XYZ.Z: 1.1800012986




Current time is: 4957
do

C:\Users\UWAdmin\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\UWAdmin\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


     power time pressure
3638   823 4957        7
         se      mean
1 0.7966157 0.7514182
Expected value (upper bound):
0.751418202307775 (1.54803387345452)
136
#####################

LINE 14: DEFINING MBO EXPERIMENT
====== MOVE LINE: 1 ======
====== SAVE LINE: 15 ======
XYZ.X: 3.4000016418
XYZ.Y: 7.4999083572
XYZ.Z: 1.1800012986




Current time is: 4957
done
done
done

 Pressure is now :  120 

   ramptime  rampdist  rampvel
0  0.242082  0.097673  0.40347
1  4.981785  2.010000  0.40347
2  0.242082  0.097673  0.00000
Total time: 5.465948999999999, Mirror_sleep: -0.257918 in MAIN
Power is now: 823 mW
XYZ.X: 2.1026356218
XYZ.Y: 7.4999968542
XYZ.Z: 1.1800012986



MIRROR DOWN
###### Starting pattern in 5 seconds! #########
MIRROR THREAD
0.24208200000000002
mirror on
MIRROR UP
MOTION THREAD

XPS run trajectory
0:00:00.281000
MIRROR DOWN
mirror off
Total time 5.465948999999999 in class
TOTAL_TIME DONE, LASER 0.05W
finished and current position is:

XYZ.X: 4.307679972
XYZ.Y: 7.499996854